# Multiple Molecules Without Sockets

Replicate the superradiance tutorial using embedded TLS drivers (`driver="tls"`).

## 1. Build the Molecule List

Each molecule is instantiated in non-socket mode with scaled dipole moments.

In [ ]:
import numpy as np
import maxwelllink as mxl

try:
    import meep as mp
except ImportError as exc:  # pragma: no cover - docs example
    raise RuntimeError("Meep is required for this tutorial. Install it via conda.") from exc

n_tls = 10
molecules = []
for _ in range(n_tls):
    molecules.append(
        mxl.Molecule(
            driver="tls",
            resolution=10,
            center=mp.Vector3(0, 0, 0),
            size=mp.Vector3(1, 1, 1),
            sigma=0.1,
            dimensions=2,
            driver_kwargs=dict(
                omega=0.242,
                mu12=187 / np.sqrt(n_tls),
                orientation=2,
                pe_initial=1e-2,
            ),
        )
    )


## 2. Launch the Embedded Simulation

In [ ]:
sim = mxl.MeepSimulation(
    molecules=molecules,
    cell_size=mp.Vector3(8, 8, 0),
    boundary_layers=[mp.PML(3.0)],
    resolution=10,
    time_units_fs=0.1,
)

sim.run(until=90)

population = np.array([entry["Pe"] for entry in molecules[0].additional_data_history])
time_au = np.array([entry["time_au"] for entry in molecules[0].additional_data_history])


## 3. Analyze the Collective Decay

the cell mirrors `tests/test_tls/test_meep_2d_tlsmolecule_n_relaxation.py`.

In [ ]:
time_meep = time_au * 0.02418884254 / 0.1
initial = population[0]
dipole_moment = 0.1 / np.sqrt(n_tls)
frequency = 1.0

single_gamma = dipole_moment**2 * frequency**2 / 2.0
collective_gamma = single_gamma * n_tls
reference = np.exp(-time_meep * collective_gamma) / (
    np.exp(-time_meep * collective_gamma) + (1.0 - initial) / initial
)

std_rel = np.std(population - reference) / initial
max_rel = np.max(np.abs(population - reference)) / initial
print(f"std_dev={std_rel:.3e}, max_abs_diff={max_rel:.3e}")

import matplotlib.pyplot as plt

plt.figure(figsize=(6, 4))
plt.plot(time_meep, population, label="Simulation")
plt.plot(time_meep, reference, label="Analytical", linestyle="--")
plt.xlabel("time (Meep units)")
plt.ylabel("excited-state population")
plt.legend()
plt.tight_layout()
plt.show()
